<a href="https://colab.research.google.com/github/yanamalaanusha/anusha-icp1-spring-2021/blob/main/KDMicp5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install pyspark

In [29]:
from __future__ import print_function
from pyspark import SparkConf, SparkContext
from pyspark.ml.feature import HashingTF, IDF, Tokenizer
from pyspark.sql import SparkSession
from pyspark.ml.feature import NGram
from pyspark.ml.feature import Word2Vec

In [30]:
# creating spark session
spark = SparkSession.builder.appName("TfIdf Example").getOrCreate()

In [31]:
# creating spark dataframe wiht the input data. You can also read the data from file. label represents the 3 documnets (0.0,0.1,0.2)
sentenceData = spark.createDataFrame([
        (0.0, "Welcome to KDM TF_IDF Tutorial."),
        (0.1, "Learn Spark ml tf_idf in today's lab."),
        (0.2, "Spark Mllib has TF-IDF.")
    ], ["label", "sentence"])

In [32]:
# creating tokens/words from the sentence data
tokenizer = Tokenizer(inputCol="sentence", outputCol="words")
wordsData = tokenizer.transform(sentenceData)

In [33]:
wordsData.show()

+-----+--------------------+--------------------+
|label|            sentence|               words|
+-----+--------------------+--------------------+
|  0.0|Welcome to KDM TF...|[welcome, to, kdm...|
|  0.1|Learn Spark ml tf...|[learn, spark, ml...|
|  0.2|Spark Mllib has T...|[spark, mllib, ha...|
+-----+--------------------+--------------------+



In [34]:
# applying tf on the words data
hashingTF = HashingTF(inputCol="words", outputCol="rawFeatures", numFeatures=20)
featurizedData = hashingTF.transform(wordsData)
# alternatively, CountVectorizer can also be used to get term frequency vectors

In [35]:
# calculating the IDF
idf = IDF(inputCol="rawFeatures", outputCol="features")
idfModel = idf.fit(featurizedData)
rescaledData = idfModel.transform(featurizedData)

In [36]:
#displaying the results
rescaledData.select("label", "features").show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(20,[2,8,13,15,17...|
|  0.1|(20,[2,3,6,7],[0....|
|  0.2|(20,[6,14,15],[0....|
+-----+--------------------+



In [37]:
spark2 = SparkSession.builder.appName("Ngram Example").getOrCreate()

In [38]:
#creating dataframe of input
wordDataFrame = spark2.createDataFrame([
    (0, ["Hi", "I", "heard", "about", "Spark"]),
    (1, ["I", "wish", "Java", "could", "use", "case", "classes"]),
    (2, ["Logistic", "regression", "models", "are", "neat"])
], ["id", "words"])


In [39]:
#creating NGrams with n=2 (two words)
ngram = NGram(n=2, inputCol="words", outputCol="ngrams")
ngramDataFrame = ngram.transform(wordDataFrame)

In [40]:
# displaying the results
ngramDataFrame.select("ngrams").show(truncate=False)

+------------------------------------------------------------------+
|ngrams                                                            |
+------------------------------------------------------------------+
|[Hi I, I heard, heard about, about Spark]                         |
|[I wish, wish Java, Java could, could use, use case, case classes]|
|[Logistic regression, regression models, models are, are neat]    |
+------------------------------------------------------------------+



In [41]:
# creating spark session
spark3 = SparkSession.builder.appName("Word2Vec Example").getOrCreate()

In [42]:
# Input data: Each row is a bag of words from a sentence or document.
documentDF = spark3.createDataFrame([
    ("McCarthy was asked to analyse the data from the first phase of trials of the vaccine.".split(" "), ),
    ("We have amassed the raw data and are about to begin analysing it.".split(" "), ),
    ("Without more data we cannot make a meaningful comparison of the two systems.".split(" "), ),
    ("Collecting data is a painfully slow process.".split(" "), ),
    ("You need a long series of data to be able to discern such a trend.".split(" "), )
], ["text"])

In [43]:
# Learn a mapping from words to Vectors.
word2Vec = Word2Vec(vectorSize=3, minCount=0, inputCol="text", outputCol="result")
model = word2Vec.fit(documentDF)
result = model.transform(documentDF)

In [44]:
for row in result.collect():
    text, vector = row
    #printing the results
    print("Text: [%s] => \nVector: %s\n" % (", ".join(text), str(vector)))

Text: [McCarthy, was, asked, to, analyse, the, data, from, the, first, phase, of, trials, of, the, vaccine.] => 
Vector: [0.022891053871717304,0.002649051835760474,0.018310461320652394]

Text: [We, have, amassed, the, raw, data, and, are, about, to, begin, analysing, it.] => 
Vector: [-0.026372590818657327,-0.01868619819959769,0.04482671529359113]

Text: [Without, more, data, we, cannot, make, a, meaningful, comparison, of, the, two, systems.] => 
Vector: [-0.01645793221317805,-0.03237590100616217,0.01814235510895602]

Text: [Collecting, data, is, a, painfully, slow, process.] => 
Vector: [0.011914271329130444,0.027699152406837256,-0.00393283592503784]

Text: [You, need, a, long, series, of, data, to, be, able, to, discern, such, a, trend.] => 
Vector: [0.05923219447334607,0.009327551970879236,0.0451309525136215]



In [45]:
# showing the synonyms and cosine similarity of the word in input data
synonyms = model.findSynonyms("data", 5)   # its okay for certain words , real bad for others
synonyms.show(5)

+--------+------------------+
|    word|        similarity|
+--------+------------------+
|McCarthy|0.9495655298233032|
|   asked|0.8464929461479187|
|vaccine.|0.8331092596054077|
|  trend.|0.7884795069694519|
| analyse| 0.706946611404419|
+--------+------------------+



In [46]:
#closing the spark sessions
spark.stop()
spark2.stop()
spark3.stop()

Creating 5 separate text files containing text data (blogs,news articles etc)

In [49]:
with open("/content/text1.txt","r+") as t1:
    doc1 = t1.read()
with open("/content/text2.txt","r+") as t2:
    doc2 = t2.read()
with open("/content/text3.txt","r+") as t3:
    doc3 = t3.read()
with open("/content/text4.txt","r+") as t4:
    doc4 = t4.read()
with open("/content/text5.txt","r+") as t5:
    doc5 = t5.read()
# Read all 5 txt files which contains news articles
documents = [doc1,doc2,doc3,doc4,doc5]

a.Find out the top10 TF-IDF words for the above input.

In [50]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

# using sklearn library which has inbuilt Tfidf vectorizer class which can generate tfidf for given corpus
vect = TfidfVectorizer()
#created TfidfVectorizer object
tfidf_matrix = vect.fit_transform(documents)
#passed list of documents or corpus to obt method fit_transform
df = pd.DataFrame(tfidf_matrix.toarray(), columns = vect.get_feature_names())
# converted method output to panda data frame 
pd.set_option('display.max_columns', 20)

df.loc['Total'] = df.sum() # adding row to value total

#filtering values of words whos tfidf is greater than 0.3
# also used transpose function here to filter out words (which was rows) and then converted matrix back to original version
print (df.T.sort_values('Total', ascending=True).tail(10).T)

             to    python        is   science       the        or  knowledge  \
0      0.000000  0.412456  0.098269  0.000000  0.000000  0.000000   0.000000   
1      0.099483  0.000000  0.070783  0.239692  0.239692  0.000000   0.000000   
2      0.197250  0.000000  0.140345  0.237625  0.000000  0.000000   0.118812   
3      0.108909  0.000000  0.077490  0.000000  0.262403  0.000000   0.524806   
4      0.000000  0.000000  0.060893  0.000000  0.000000  0.511163   0.000000   
Total  0.405642  0.412456  0.447780  0.477317  0.502095  0.511163   0.643618   

             of      data       and  
0      0.116185  0.000000  0.116185  
1      0.334754  0.000000  0.251065  
2      0.000000  0.594062  0.331866  
3      0.091618  0.000000  0.274853  
4      0.215985  0.206201  0.000000  
Total  0.758542  0.800263  0.973970  


b.Find out the top10 TF-IDF words for the lemmatized input

In [51]:
import nltk;nltk.download('punkt');nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

words1 = nltk.word_tokenize(doc1)
words2 = nltk.word_tokenize(doc2)
words3 = nltk.word_tokenize(doc3)
words4 = nltk.word_tokenize(doc4)
words5 = nltk.word_tokenize(doc5)

lemmatized_document1 = ' '.join([lemmatizer.lemmatize(w) for w in words1])
lemmatized_document2 = ' '.join([lemmatizer.lemmatize(w) for w in words2])
lemmatized_document3 = ' '.join([lemmatizer.lemmatize(w) for w in words3])
lemmatized_document4 = ' '.join([lemmatizer.lemmatize(w) for w in words4])
lemmatized_document5 = ' '.join([lemmatizer.lemmatize(w) for w in words5])

documents = [lemmatized_document1,lemmatized_document2,lemmatized_document3,lemmatized_document4,lemmatized_document5]

# using sklearn library which has inbuilt Tfidf vectorizer class which can generate tfidf for given corpus
vect = TfidfVectorizer()
#created TfidfVectorizer object
tfidf_matrix = vect.fit_transform(documents)
#passed list of documents or corpus to obt method fit_transform
df = pd.DataFrame(tfidf_matrix.toarray(), columns = vect.get_feature_names())
# converted method output to panda data frame 

df.loc['Total'] = df.sum() # adding row to value total

#filtering values of words whos tfidf is greater than 0.3
# also used transpose function here to filter out words (which was rows) and then converted matrix back to original version
print (df.T.sort_values('Total', ascending=True).tail(10).T)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
         python   machine        is   science        or       the  knowledge  \
0      0.412456  0.000000  0.098269  0.000000  0.000000  0.000000   0.000000   
1      0.000000  0.097930  0.069678  0.235951  0.000000  0.235951   0.000000   
2      0.000000  0.098842  0.140654  0.238148  0.000000  0.000000   0.119074   
3      0.000000  0.220188  0.078333  0.000000  0.000000  0.265258   0.530516   
4      0.000000  0.000000  0.058997  0.000000  0.495243  0.000000   0.000000   
Total  0.412456  0.416961  0.445931  0.474099  0.495243  0.501209   0.649590   

             of     data       and  
0      0.116185  0.00000  0.116185  
1      0.329529  0.00000  0.247146  
2      0.000000  0.59537  0.332597  
3      0.092615  0.00000  0.277844  
4      0.209259  0.19978  0.

c.Find out the top10TF-IDF words for the n-gram based input.

In [52]:
# this function takes document and n int value to generate list of n grams
def ngrams(input, n):
    input = input.split(' ')
    output = []
    for i in range(len(input)-n+1):
        output.append(input[i:i+n])
    return output

ngram_doc1 = ' '.join([' '.join(x) for x in ngrams(doc1, 3)])
ngram_doc2 = ' '.join([' '.join(x) for x in ngrams(doc2, 3)])
ngram_doc3 = ' '.join([' '.join(x) for x in ngrams(doc3, 3)])
ngram_doc4 = ' '.join([' '.join(x) for x in ngrams(doc4, 3)])
ngram_doc5 = ' '.join([' '.join(x) for x in ngrams(doc5, 3)])

# documents = [ngram_doc1,ngram_doc2,ngram_doc3,ngram_doc4,ngram_doc5]

documents = [doc1,doc2,doc3,doc4,doc5]

# using sklearn library which has inbuilt Tfidf vectorizer class which can generate tfidf for given corpus
vect = TfidfVectorizer( ngram_range=(3,3)) # TfidfVectorizer has inbuilt ngram kwarg which show tfidf for ngrams
#created TfidfVectorizer object
tfidf_matrix = vect.fit_transform(documents)
#passed list of documents or corpus to obt method fit_transform
df = pd.DataFrame(tfidf_matrix.toarray(), columns = vect.get_feature_names())
# converted method output to panda data frame 

df.loc['Total'] = df.sum() # adding row to value total

#filtering values of words whos tfidf is greater than 0.3
# also used transpose function here to filter out words (which was rows) and then converted matrix back to original version
print (df.T.sort_values('Total', ascending=True).tail(10).T)

       level and general  notable use of  of significant indentation  \
0               0.213201        0.213201                    0.213201   
1               0.000000        0.000000                    0.000000   
2               0.000000        0.000000                    0.000000   
3               0.000000        0.000000                    0.000000   
4               0.000000        0.000000                    0.000000   
Total           0.213201        0.213201                    0.213201   

       philosophy emphasizes code  programming language python  \
0                        0.213201                     0.213201   
1                        0.000000                     0.000000   
2                        0.000000                     0.000000   
3                        0.000000                     0.000000   
4                        0.000000                     0.000000   
Total                    0.213201                     0.213201   

       purpose programming langu

2.Write a simple spark program to read a dataset and find the W2V similar words (words with higher cosine similarity) for the Top10 TF-IDF Words

In [53]:
from __future__ import print_function
from pyspark import SparkConf, SparkContext
from pyspark.ml.feature import HashingTF, IDF, Tokenizer
from pyspark.sql import SparkSession
from pyspark.ml.feature import NGram
from pyspark.ml.feature import Word2Vec
# creating spark session
spark = SparkSession.builder.appName("TfIdf Example").getOrCreate()

documentData = spark.createDataFrame([
        (0.0, doc1),
        (0.1, doc2),
        (0.2, doc3),
        (0.3, doc4),
        (0.5, doc5)
    ], ["label", "document"])

# creating tokens/words from the sentence data
tokenizer = Tokenizer(inputCol="document", outputCol="words")
wordsData = tokenizer.transform(documentData)
print (documentData)
wordsData.show()

DataFrame[label: double, document: string]
+-----+--------------------+--------------------+
|label|            document|               words|
+-----+--------------------+--------------------+
|  0.0|Python is an inte...|[python, is, an, ...|
|  0.1|Computer science ...|[computer, scienc...|
|  0.2|Data science is a...|[data, science, i...|
|  0.3|Knowledge extract...|[knowledge, extra...|
|  0.5|Data are characte...|[data, are, chara...|
+-----+--------------------+--------------------+



a.Try without NLP

In [54]:
# applying tf on the words data
hashingTF = HashingTF(inputCol="words", outputCol="rawFeatures", numFeatures=200)
tf = hashingTF.transform(wordsData)
# alternatively, CountVectorizer can also be used to get term frequency vectors
# calculating the IDF
tf.cache()
idf = IDF(inputCol="rawFeatures", outputCol="features")
idf = idf.fit(tf)
tfidf = idf.transform(tf)
#displaying the results
tfidf.select("label", "features").show()


print("TF-IDF without NLP:")
for each in tfidf.collect():
    print(each)
    print(each['rawFeatures'])
spark.stop()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(200,[9,20,21,43,...|
|  0.1|(200,[9,11,16,17,...|
|  0.2|(200,[9,17,19,24,...|
|  0.3|(200,[9,11,17,23,...|
|  0.5|(200,[3,6,9,15,24...|
+-----+--------------------+

TF-IDF without NLP:
Row(label=0.0, document="Python is an interpreted, high-level and general-purpose programming language. Python's design philosophy emphasizes code readability with its notable use of significant indentation.", words=['python', 'is', 'an', 'interpreted,', 'high-level', 'and', 'general-purpose', 'programming', 'language.', "python's", 'design', 'philosophy', 'emphasizes', 'code', 'readability', 'with', 'its', 'notable', 'use', 'of', 'significant', 'indentation.'], rawFeatures=SparseVector(200, {9: 1.0, 20: 1.0, 21: 1.0, 43: 1.0, 50: 1.0, 67: 1.0, 73: 1.0, 80: 1.0, 90: 1.0, 91: 1.0, 95: 1.0, 109: 1.0, 110: 1.0, 119: 1.0, 120: 1.0, 122: 1.0, 128: 1.0, 133: 1.0, 152: 1.0, 163: 1.0, 179: 1.0, 198: 1.0}), features=S


b.Try with Lemmatization

In [55]:
import nltk;nltk.download('punkt');nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

words1 = nltk.word_tokenize(doc1)
words2 = nltk.word_tokenize(doc2)
words3 = nltk.word_tokenize(doc3)
words4 = nltk.word_tokenize(doc4)
words5 = nltk.word_tokenize(doc5)

lemmatized_document1 = ' '.join([lemmatizer.lemmatize(w) for w in words1])
lemmatized_document2 = ' '.join([lemmatizer.lemmatize(w) for w in words2])
lemmatized_document3 = ' '.join([lemmatizer.lemmatize(w) for w in words3])
lemmatized_document4 = ' '.join([lemmatizer.lemmatize(w) for w in words4])
lemmatized_document5 = ' '.join([lemmatizer.lemmatize(w) for w in words5])

### lemmatizing words from 5 input docs same as previos task

# creating spark session
spark = SparkSession.builder.appName("TfIdf Example").getOrCreate()

documentData = spark.createDataFrame([
        (0.0, lemmatized_document1),
        (0.1, lemmatized_document2),
        (0.2, lemmatized_document3),
        (0.3, lemmatized_document4),
        (0.5, lemmatized_document5)
    ], ["label", "document"])

# creating tokens/words from the sentence data
tokenizer = Tokenizer(inputCol="document", outputCol="words")
wordsData = tokenizer.transform(documentData)
print (documentData)
wordsData.show()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
DataFrame[label: double, document: string]
+-----+--------------------+--------------------+
|label|            document|               words|
+-----+--------------------+--------------------+
|  0.0|Python is an inte...|[python, is, an, ...|
|  0.1|Computer science ...|[computer, scienc...|
|  0.2|Data science is a...|[data, science, i...|
|  0.3|Knowledge extract...|[knowledge, extra...|
|  0.5|Data are characte...|[data, are, chara...|
+-----+--------------------+--------------------+



In [56]:
# applying tf on the words data
hashingTF = HashingTF(inputCol="words", outputCol="rawFeatures", numFeatures=200)
tf = hashingTF.transform(wordsData)
# alternatively, CountVectorizer can also be used to get term frequency vectors
# calculating the IDF
tf.cache()
idf = IDF(inputCol="rawFeatures", outputCol="features")
idf = idf.fit(tf)
tfidf = idf.transform(tf)
#displaying the results
tfidf.select("label", "features").show()


print("TF-IDF with Lemmatization:")
for each in tfidf.collect():
    print(each)
    print(each['rawFeatures'])
spark.stop()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(200,[9,20,21,28,...|
|  0.1|(200,[9,16,17,24,...|
|  0.2|(200,[9,15,17,28,...|
|  0.3|(200,[9,11,17,23,...|
|  0.5|(200,[3,9,11,15,2...|
+-----+--------------------+

TF-IDF with Lemmatization:
Row(label=0.0, document="Python is an interpreted , high-level and general-purpose programming language . Python 's design philosophy emphasizes code readability with it notable use of significant indentation .", words=['python', 'is', 'an', 'interpreted', ',', 'high-level', 'and', 'general-purpose', 'programming', 'language', '.', 'python', "'s", 'design', 'philosophy', 'emphasizes', 'code', 'readability', 'with', 'it', 'notable', 'use', 'of', 'significant', 'indentation', '.'], rawFeatures=SparseVector(200, {9: 1.0, 20: 1.0, 21: 1.0, 28: 2.0, 50: 1.0, 63: 1.0, 67: 1.0, 73: 1.0, 86: 1.0, 90: 1.0, 91: 1.0, 95: 1.0, 109: 2.0, 110: 1.0, 116: 1.0, 119: 1.0, 120: 1.0, 122: 1.0, 133: 1.0, 152: 1.0, 163: 1.0

c.Try with NGrams

In [57]:
spark = SparkSession.builder.appName("TfIdf Example").getOrCreate()

documentData = spark.createDataFrame([
        (0.0, doc1.split(' ')),
        (0.1, doc2.split(' ')),
        (0.2, doc3.split(' ')),
        (0.3, doc4.split(' ')),
        (0.5, doc5.split(' '))
    ], ["label", "document"])


ngram = NGram(n=2, inputCol="document", outputCol="ngrams")

ngramDataFrame = ngram.transform(documentData)

# applying tf on the words data
hashingTF = HashingTF(inputCol="ngrams", outputCol="rawFeatures", numFeatures=200)
tf = hashingTF.transform(ngramDataFrame)
# alternatively, CountVectorizer can also be used to get term frequency vectors
# calculating the IDF
tf.cache()
idf = IDF(inputCol="rawFeatures", outputCol="features")
idf = idf.fit(tf)
tfidf = idf.transform(tf)
#displaying the results
tfidf.select("label", "features").show()


print("TF-IDF with ngram:")
for each in tfidf.collect():
    print(each)
    print(each['rawFeatures'])
spark.stop()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(200,[1,9,15,20,2...|
|  0.1|(200,[4,7,9,11,14...|
|  0.2|(200,[13,14,18,19...|
|  0.3|(200,[1,10,15,16,...|
|  0.5|(200,[15,22,25,31...|
+-----+--------------------+

TF-IDF with ngram:
Row(label=0.0, document=['Python', 'is', 'an', 'interpreted,', 'high-level', 'and', 'general-purpose', 'programming', 'language.', "Python's", 'design', 'philosophy', 'emphasizes', 'code', 'readability', 'with', 'its', 'notable', 'use', 'of', 'significant', 'indentation.'], ngrams=['Python is', 'is an', 'an interpreted,', 'interpreted, high-level', 'high-level and', 'and general-purpose', 'general-purpose programming', 'programming language.', "language. Python's", "Python's design", 'design philosophy', 'philosophy emphasizes', 'emphasizes code', 'code readability', 'readability with', 'with its', 'its notable', 'notable use', 'use of', 'of significant', 'significant indentation.'], rawFeatures=SparseVector(2